The experiment was raised by the ICLR2022 reviewers.

We aim to evaluate the methods in different experiment settings.

In [ ]:
import os, sys
import pandas as pd
import wandb
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import display

In [ ]:
sns.set_style("ticks")
cmap = sns.color_palette()
sns.set_palette(sns.color_palette())

In [ ]:
cache_path = './fig/flops_acc_curve'
if not os.path.exists(cache_path):
    os.makedirs(cache_path)

In [ ]:
data = 'Digits'

sweep_dict = {
    'FedAvg': "jyhong/SplitMix_release/sweeps/8g8s7kp4",
    'SHeteroFL': "jyhong/SplitMix_release/sweeps/0lh7d73x",
    'SHeteroFL vary budgets': "jyhong/SplitMix_release/sweeps/jbak4jzs",
    'HeteroFL ln': "jyhong/SplitMix_release/sweeps/a36ramy7",
    'SplitMix': "jyhong/SplitMix_release/sweeps/3wr7bsxb",
    'SplitMix vary budget': "jyhong/SplitMix_release/sweeps/8g0irs68",
    'SplitMix ln': "jyhong/SplitMix_release/sweeps/wz10puq8",
}

In [ ]:
agg_df_dict = {}

In [ ]:
def get_slimmabe_ratios(mode: str):
    ps = mode.split('-')
    slimmable_ratios = []
    for p in ps:
        if 'd' in p:
            p, q = p.split('d')  # p: 1/p-net; q: weight of the net in samples
            p, q = int(p), int(q)
            p = p * 1. / q
        else:
            p = int(p)
        slimmable_ratios.append(1. / p)
#     print(f"Set slim ratios: {self.slimmable_ratios} by mode: {mode}")
    return slimmable_ratios

In [ ]:
def fetch_config_summary(runs, config_keys, summary_keys):
    df_dict = defaultdict(list)
    for run in runs:
        if run.state != 'finished':
            print("WARN: run not finished yet")
        history_len = 0
        missing_sum_key = []
        for k in summary_keys:
            if k in run.summary:
                h = run.summary[k]
                df_dict[k].append(h)
            else:
                missing_sum_key.append(k)
                break
        if len(missing_sum_key) > 0:
            print(f"missing key: {missing_sum_key}")
            continue
        for k in config_keys:
            df_dict[k].append(run.config[k])
    return df_dict

## FedAvg

In [ ]:
mode = 'FedAvg'
api = wandb.Api()
sweep = api.sweep(sweep_dict[mode])

In [ ]:
df_dict = fetch_config_summary(
    sweep.runs,
    config_keys = ['width_scale'], 
    summary_keys = ['avg test acc', 'GFLOPs', 'model size (MB)']
)
df = pd.DataFrame(df_dict)
df['mode'] = mode
df['width_scale'] = df['width_scale'] * 100
df['width'] = df['width_scale']
df['slim_ratios'] = 'w/o constraint'

agg_df_dict[mode] = df  # [df['slim_ratio'] == 1.0]

In [ ]:
fig, ax = plt.subplots(1, 1)
# for slim_ratio, val_accs in zip(df_dict['slim_ratio'], df_dict['val_acc']):
#     plt.plot(val_accs)
sns.lineplot(data=df, x='width', y='avg test acc', marker='o')
ax.set(xticks=df['width'].unique())
# ax.set(xlim=(0, 150), ylim=(0.3, 0.9))
ax.grid(True)

## SHeteroFL

In [ ]:
mode = 'SHeteroFL'
api = wandb.Api()
sweep = api.sweep(sweep_dict[mode])

In [ ]:
df_dict = fetch_config_summary(
    sweep.runs,
    config_keys = ['test_slim_ratio'], 
    summary_keys = ['avg test acc', 'GFLOPs', 'model size (MB)']
)
df = pd.DataFrame(df_dict)
df['test_slim_ratio'] = df['test_slim_ratio'] * 100
df['width'] = df['test_slim_ratio']
df['slim_ratios'] = '8-4-2-1'

# df['mode'] = mode
# agg_df_dict[mode] = df[df['slim_sch'] == 'group_size']

df['mode'] = mode
agg_df_dict[mode] = df
# agg_df_dict['S'+mode] = df[df['slim_sch'] == 'group_slimmable']

In [ ]:
fig, ax = plt.subplots(1, 1)
# for slim_ratio, val_accs in zip(df_dict['slim_ratio'], df_dict['val_acc']):
#     plt.plot(val_accs)
sns.lineplot(data=df, x='test_slim_ratio', y='avg test acc', 
             marker='o')
ax.set(xticks=df['test_slim_ratio'].unique())
# ax.set(xlim=(0, 150), ylim=(0.3, 0.9))
ax.grid(True)

In [ ]:
mode = 'SHeteroFL vary budgets'
api = wandb.Api()
sweep = api.sweep(sweep_dict[mode])

df_dict = fetch_config_summary(
    sweep.runs,
    config_keys = ['test_slim_ratio', 'slim_ratios'], 
    summary_keys = ['avg test acc', 'GFLOPs', 'model size (MB)']
)
del_idxs = []
for idx in range(len(df_dict['slim_ratios'])):
    slim_ratios = get_slimmabe_ratios(df_dict['slim_ratios'][idx])
    # print(df_dict['slim_ratios'][idx], slim_ratios)
    if df_dict['test_slim_ratio'][idx] not in slim_ratios:
        # print("del", idx, df_dict['test_slim_ratio'][idx])
        del_idxs.append(idx)
for k in df_dict:
    df_dict[k] = [v for i, v in enumerate(df_dict[k]) if i not in del_idxs]
df = pd.DataFrame(df_dict)
df['test_slim_ratio'] = df['test_slim_ratio'] * 100
df['width'] = df['test_slim_ratio']

df['mode'] = 'SHeteroFL'
agg_df_dict[mode] = df  # [df['slim_sch'] == 'group_slimmable']

In [ ]:
sns.lineplot(data=df, x='width', y='avg test acc', hue='slim_ratios', marker='o')
plt.grid(True)

## Split-Mix

In [ ]:
dfs = []
# for atom_slim_ratio in [0.125, 0.25]:
for mode in ['SplitMix']:  # , 'SplitMix incr']:
    print(f"mode: {mode}")
    api = wandb.Api()
    sweep = api.sweep(sweep_dict[mode])

    df_dict = fetch_config_summary(
        sweep.runs,
        config_keys = ['test_slim_ratio', 'atom_slim_ratio'], 
        summary_keys = ['avg test acc', 'GFLOPs', 'model size (MB)']
    )
    df = pd.DataFrame(df_dict)
    df['mode'] = mode
    df['test_slim_ratio'] = df['test_slim_ratio'] * 100
    df['width'] = df['test_slim_ratio']
    df['slim_ratios'] = '8-4-2-1'
    dfs.append(df)
    agg_df_dict[mode] = df
    
df = pd.concat(dfs)

In [ ]:
fig, ax = plt.subplots(1, 1)
# for slim_ratio, val_accs in zip(df_dict['slim_ratio'], df_dict['val_acc']):
#     plt.plot(val_accs)
sns.lineplot(data=df, x='test_slim_ratio', y='avg test acc', marker='o')
ax.set(xticks=df['test_slim_ratio'].unique())
# ax.set(xlim=(0, 150), ylim=(0.3, 0.9))
ax.grid(True)

In [ ]:
for mode in ['SplitMix vary budget']:
    # 'SplitMix step=0.25 non-exp'
    api = wandb.Api()
    sweep = api.sweep(sweep_dict[mode])

    print(f"mode: {mode}")
    api = wandb.Api()
    sweep = api.sweep(sweep_dict[mode])

    df_dict = fetch_config_summary(
        sweep.runs,
        config_keys = ['test_slim_ratio', 'atom_slim_ratio', 'slim_ratios'], 
        summary_keys = ['avg test acc', 'GFLOPs', 'model size (MB)']
    )
    df = pd.DataFrame(df_dict)
    df['mode'] = 'SplitMix'
    df['test_slim_ratio'] = df['test_slim_ratio'] * 100
    df['width'] = df['test_slim_ratio']
    df = df[df['slim_ratios'] != '8-4-2-1']
    agg_df_dict[mode] = df

In [ ]:
fig, ax = plt.subplots(1, 1)
# for slim_ratio, val_accs in zip(df_dict['slim_ratio'], df_dict['val_acc']):
#     plt.plot(val_accs)
sns.lineplot(data=df, x='width', y='avg test acc', marker='o', hue='slim_ratios')
ax.set(xticks=df['test_slim_ratio'].unique())
# ax.set(xlim=(0, 150), ylim=(0.3, 0.9))
ax.grid(True)

## Aggregation

In [ ]:
agg = pd.concat([v for k, v in agg_df_dict.items()])

In [ ]:
cmap = sns.color_palette(as_cmap=True)
len(cmap)

more budget-sufficient clients

In [ ]:
agg = pd.concat([v for k, v in agg_df_dict.items()])
agg = agg.reset_index()
agg['avg test acc'] = agg['avg test acc'] * 100
agg['MFLOPs'] = agg['GFLOPs'] * 1e3
agg['method'] = agg['mode'].apply(lambda n: n if n != 'FedAvg' else 'Ind. FedAvg')
agg['budgets'] = agg['slim_ratios'].apply(lambda n: (n.replace('d', '/')) if '-' in n else n)
agg = agg[agg['slim_ratios'].apply(lambda n: 'd' not in n)]

fig, ax = plt.subplots(1, 1, figsize=(5,3))
sns.lineplot(data=agg, x='width', y='avg test acc', marker='o', style='method', hue='budgets',
            style_order=['Ind. FedAvg', 'SplitMix', 'SHeteroFL'], palette=cmap[:len(agg['budgets'].unique())])
ax.set(xticks=agg['width'].unique(), ylabel='average test accuracy (%)',
      xlabel='width (%)')
# ax.set(xlim=(None, 200))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid(True)

plt.tight_layout()
out_file = os.path.join(cache_path, f'vary_budget_dist_skew.pdf')
print(f"save fig => {out_file}")
plt.savefig(out_file)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4,3))
df = agg[np.isin(agg['budgets'], ['8-4-2-1', 'w/o constraint'])]
# df = df[np.isin(df['method'], ['Ind. FedAvg', 'SHeteroFL'])]
sns.lineplot(data=df, x='width', y='avg test acc', marker='o', style='method', # hue='budgets',
#             style_order=['Ind. FedAvg', 'SplitMix', 'SHeteroFL'], # palette=cmap[:len(df['budgets'].unique())]
            )
ax.set(xticks=df['width'].unique(), ylabel='average test accuracy (%)', xlabel='width (%)')
ax.set(ylim=(81, 90.2))
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# ax.legend('')
ax.grid(True)

In [ ]:
df_dict = defaultdict(list)
for slim_ratio in agg['slim_ratios']:
    if slim_ratio != 'w/o constraint':
        slim_ratios = get_slimmabe_ratios(slim_ratio)
        df_dict['group'].extend([1,2,3,4])
        df_dict['width constraint'].extend(slim_ratios)
        df_dict['budgets'].extend([slim_ratio]*len(slim_ratios))
df = pd.DataFrame(df_dict)
# df

fig, ax = plt.subplots(1, 1, figsize=(5,3))
sns.barplot(data=df, x='group', y='width constraint', hue='budgets', 
            palette=cmap[1:len(agg['budgets'].unique())])
plt.grid(True)

plt.tight_layout()
out_file = os.path.join(cache_path, f'budget_dist_skew.pdf')
print(f"save fig => {out_file}")
plt.savefig(out_file)

plt.show()

In [ ]:
df_ = df[df['budgets'] == '8-4-2-1']
fig, ax = plt.subplots(1, 1, figsize=(5,3))
sns.barplot(data=df_, x='group', y='width constraint', hue='budgets')
plt.grid(True)

step-increase budgets

In [ ]:
agg = pd.concat([v for k, v in agg_df_dict.items()])
agg = agg.reset_index()
agg['avg test acc'] = agg['avg test acc'] * 100
agg['MFLOPs'] = agg['GFLOPs'] * 1e3
agg['method'] = agg['mode'].apply(lambda n: n if n != 'FedAvg' else 'Ind. FedAvg')
agg['budgets'] = agg['slim_ratios'].apply(lambda n: (n.replace('d', '/')) if '-' in n else n)
agg = agg[agg['budgets'].apply(lambda n: '/' in n)]

fig, ax = plt.subplots(1, 1, figsize=(5,3))
sns.lineplot(data=agg, x='width', y='avg test acc', marker='o', style='method', hue='budgets',
            style_order=['Ind. FedAvg', 'SplitMix', 'SHeteroFL'])
ax.set(xticks=agg['width'].unique(), ylabel='average test accuracy (%)', ylim=(77.5, None))
ax.set(xlabel='width (%)')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid(True)

plt.tight_layout()
out_file = os.path.join(cache_path, f'vary_budget_dist_step.pdf')
print(f"save fig => {out_file}")
plt.savefig(out_file)

plt.show()

In [ ]:
df_dict = defaultdict(list)
for slim_ratio in agg['slim_ratios']:
    if slim_ratio != 'w/o constraint':
        slim_ratios = get_slimmabe_ratios(slim_ratio)
        df_dict['group'].extend([1,2,3,4])
        df_dict['width constraint'].extend(slim_ratios)
        df_dict['budgets'].extend([slim_ratio]*len(slim_ratios))
df = pd.DataFrame(df_dict)
# df

fig, ax = plt.subplots(1, 1, figsize=(5,3))
sns.barplot(data=df, x='group', y='width constraint', hue='budgets')
plt.grid(True)

plt.tight_layout()
out_file = os.path.join(cache_path, f'budget_dist_step.pdf')
print(f"save fig => {out_file}")
plt.savefig(out_file)

plt.show()

log normal budget distribution

In [ ]:
ln_agg_df_dict = {}
for mode in ['SplitMix ln', 'HeteroFL ln']:
    # 'SplitMix step=0.25 non-exp'
    api = wandb.Api()
    sweep = api.sweep(sweep_dict[mode])

    print(f"mode: {mode}")
    api = wandb.Api()
    sweep = api.sweep(sweep_dict[mode])

    df_dict = fetch_config_summary(
        sweep.runs,
        config_keys = ['test_slim_ratio', 'slim_ratios'], 
        summary_keys = ['avg test acc', 'GFLOPs', 'model size (MB)']
    )
    df = pd.DataFrame(df_dict)
    df['mode'] = mode.split(' ')[0]
    df['test_slim_ratio'] = df['test_slim_ratio'] * 100
    df['width'] = df['test_slim_ratio']
    df = df[df['slim_ratios'] != '8-4-2-1']
    ln_agg_df_dict[mode] = df
ln_agg_df_dict['FedAvg'] = agg_df_dict['FedAvg']

In [ ]:
agg = pd.concat([v for k, v in ln_agg_df_dict.items()])
agg = agg.reset_index()
agg['avg test acc'] = agg['avg test acc'] * 100
agg['method'] = agg['mode'].apply(lambda n: n if n != 'FedAvg' else 'Ind. FedAvg')

fig, ax = plt.subplots(1, 1, figsize=(4,3))
sns.lineplot(data=agg, x='width', y='avg test acc', marker='o', style='method', #hue='budgets',
            style_order=['Ind. FedAvg', 'SplitMix', 'HeteroFL'])
ax.set(xticks=agg['width'].unique(), ylabel='average test accuracy (%)', ylim=(65, None),
      xlabel='width (%)')
ax.grid(True)

plt.tight_layout()
out_file = os.path.join(cache_path, f'vary_budget_dist_ln.pdf')
print(f"save fig => {out_file}")
plt.savefig(out_file)

plt.show()

In [ ]:
def get_slim_ratio_schedule(train_slim_ratios: list, mode: str, client_num):
    if mode.startswith('ln'):  # lognorm
        ws = sorted(train_slim_ratios)
        min_w = min(train_slim_ratios)
        from scipy.stats import lognorm
        s, scale = [float(v) for v in mode[len('ln'):].split('_')]
        rv = lognorm(s=s, scale=scale)
        print(ws)
        cdfs = [rv.cdf(w) for w in ws] + [1.]
        print(cdfs)
        qs = [c - rv.cdf(min_w) for c in cdfs]
        r = (qs[-1]-qs[0])
        qs = [int(client_num * (q-qs[0]) / r) for q in qs]
        print(qs)
        slim_ratios = np.zeros(client_num)
        for i in range(len(qs)-1):
            slim_ratios[qs[i]:qs[i+1]] = ws[i]
        return slim_ratios
get_slim_ratio_schedule(np.arange(0.125, 1.+0.125, 0.125), 'ln0.5_0.4', 50)

In [ ]:
sch = get_slim_ratio_schedule(np.arange(0.125, 1.+0.125, 0.125), 'ln0.5_0.4', 50)
budgets, cnts = np.unique(sch, return_counts=True)
print(budgets, cnts)

fig, ax = plt.subplots(1, 1, figsize=(4,3))
ax.bar(x=budgets, height=cnts, width=[0.125]*len(budgets), align='center')

ax.grid(True)
ax.set(xlabel='budget (maximal compatible width)', ylabel='number of clients')
ax.set_xticks(budgets)
ax.set_xticklabels(budgets)

plt.tight_layout()
out_file = os.path.join(cache_path, f'digits_budget_dist_ln0.5_0.4.pdf')
print(f"save fig => {out_file}")
plt.savefig(out_file)

plt.show()